## Initialize notebook

In [ ]:
import glob
import os
import re

import ipywidgets
import ipywidgets as widgets
import pandas as pd
import skimage.io

In [ ]:
%gui qt
import napari
viewer = napari.Viewer()

## Select image 👩🏻‍💻

In [ ]:
# Set analysis folder
dir_analysis = "./"

In [ ]:
# Bead alignment
alignment = skimage.io.imread(os.path.join(dir_analysis, "alignment.png"))
viewer.add_image(alignment, name="Alignment")

In [ ]:
# Select from nd files
files = sorted(glob.glob(os.path.join(dir_analysis, "preprocessed", "*.tif")))
files = sorted([os.path.basename(f).replace(".tif", "") for f in files])
file = widgets.Dropdown(
    options=files, description="File:", layout=ipywidgets.Layout(width="100%")
)
file

## Load images 🌠

In [ ]:
viewer.layers.clear()

# Images
image = skimage.io.imread(
    os.path.join(dir_analysis, "preprocessed", f"{file.value}.tif")
)
viewer.add_image(image[0], name="MS2 Channel", colormap="magenta", blending="additive")
viewer.add_image(image[1], name="SunTag Channel", colormap="green", blending="additive")

# Segmentation
segmap = skimage.io.imread(
    os.path.join(dir_analysis, "segmentation", f"{file.value}.tif")
).astype(int)
viewer.add_labels(segmap, name="Segmentation", blending="translucent")

# All tracks
track_cols = ["particle", "frame", "y", "x"]
track_ms2 = pd.read_parquet(
    os.path.join(dir_analysis, "track_ms2", f"{file.value}.parq")
)
track_sun = pd.read_parquet(
    os.path.join(dir_analysis, "track_suntag", f"{file.value}.parq")
)
viewer.add_tracks(track_ms2[track_cols], name="Track MS2", colormap="magenta")
viewer.add_tracks(track_sun[track_cols], name="Track SunTag", colormap="green")

# Colocalized vs non-colocalized (ms2 channel only)
coloc = pd.read_parquet(
    os.path.join(dir_analysis, "colocalization", f"{file.value}.parq")
)

viewer.add_tracks(
    coloc[(coloc["coloc_particle"].isna()) & (coloc["channel"] == "ms2")][track_cols],
    name="Track Coloc",
    colormap="red",
)
viewer.add_tracks(
    coloc[(~coloc["coloc_particle"].isna()) & (coloc["channel"] == "ms2")][track_cols],
    name="Track Empty",
    colormap="blue",
)